# Search Strategy Tester

Like AI_Search_tester.ipynb but streamlined

### Features
 - Load different search strategies
 - Compare search strategies
 - Visualize search strategies
 - Save results to file

### Install Packages

pip install packagers from requirements.txt if not already installed

In [9]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


### Import Packages

In [10]:
import os
import subprocess
import numpy as np
from tqdm.notebook import tqdm
from renderState import *

from searchclient.agent_types.classic import * 

from searchclient.domains.hospital.actions import (
    NoOpAction, MoveAction, PushAction, PullAction, AnyAction, DEFAULT_MAPF_ACTION_LIBRARY, DEFAULT_HOSPITAL_ACTION_LIBRARY
)

# Import state, goal description and level classes for the MAvis hospital environment
from searchclient.domains.hospital.state import HospitalState
from searchclient.domains.hospital.goal_description import HospitalGoalDescription
from searchclient.domains.hospital.level import HospitalLevel

from searchclient.search_algorithms.graph_search import graph_search

# Import the different search strategies for both uninformed and informed search
from searchclient.strategies.bfs import FrontierBFS
from searchclient.strategies.dfs import FrontierDFS
from searchclient.strategies.bestfirst import FrontierBestFirst, FrontierGreedy, FrontierAStar

# Import heuristic classes, to be used in informed search methods
from searchclient.domains.hospital.heuristics import (
    HospitalZeroHeuristic, HospitalGoalCountHeuristics, HospitalAdvancedHeuristics
)

# import functions defined in AI_Search_Experimentation_Notebook.ipynb copied over to helpers.py
from helpers import *

print("Modules imported successfully.")

Modules imported successfully.


## Parameters to adjust

In [14]:
level_path = "levels/BFSfriendly.lvl"

# for pure pathfinding problems
action_library = DEFAULT_MAPF_ACTION_LIBRARY

# for sokoban-like problems (includes Push and Pull actions)
# action_library = DEFAULT_HOSPITAL_ACTION_LIBRARY

heuristic_name = "goalcount" # zero, goalcount, advanced
strategy_name = "astar" # bfs, dfs, astar, greedy, astarverbose, greedyverbose
n_trials = 1 # number of trials

***Don't need to edit just run***

Load Level, level visualization, heuristic and action set

In [15]:
level_lines = load_level_file_from_path(level_path)
level = HospitalLevel.parse_level_lines(level_lines)

initial_state = HospitalState(level, level.initial_agent_positions, level.initial_box_positions)

goal_description = HospitalGoalDescription(level, level.box_goals + level.agent_goals)

print('The initial state of the level is:')
print(initial_state)

print('\nThe goal description of the level is:')
print(goal_description) 

action_set = [action_library] * level.num_agents

# load heuristic and action set
heuristic = {
        'zero': HospitalZeroHeuristic,
        'goalcount': HospitalGoalCountHeuristics,
        'advanced': HospitalAdvancedHeuristics,
    }.get(heuristic_name, HospitalZeroHeuristic)()

frontier = {
        'bfs': FrontierBFS,
        'dfs': FrontierDFS,
        'astar': lambda: FrontierAStar(heuristic),
        'greedy': lambda: FrontierGreedy(heuristic),
        'astarverbose': lambda: FrontierAStar(heuristic, verbose=True),
        'greedyverbose': lambda: FrontierGreedy(heuristic, verbose=True)
    }.get(strategy_name, FrontierBFS)()

The initial state of the level is:
++++++++++++++++
+              +
+              +
+              +
+      01      +
+              +
+              +
+              +
++++++++++++++++

The goal description of the level is:
((3, 8), '0', True) and ((3, 9), '1', True)


## Run search algorithm

In [18]:
plans, sol_lengths, generated, elapsed = [], [], [], []
for n in tqdm(range(n_trials)):
    planning_success, plan, num_generated, elapsed_time = graph_search(initial_state, action_set, goal_description, frontier)
    plans.append(plan)
    sol_lengths.append(len(plan))
    generated.append(int(num_generated))
    elapsed.append(elapsed_time)

  0%|          | 0/1 [00:00<?, ?it/s]

### Trial results

In [19]:
# The graph search function returns the following:
print('Number of trials:', n_trials)
print('Average solution length:', np.mean(sol_lengths))
print('Solution length variance :', np.var(sol_lengths))
print('Average number of states generated:', np.mean(generated))
print('Number of states generated variance :', np.var(generated))

Number of trials: 1
Average solution length: 2.0
Solution length variance : 0.0
Average number of states generated: 3.0
Number of states generated variance : 0.0


### Visualize trial results

In [7]:
# Select trial to render
index_to_render = 0

render_plan(
    level_path,
    plans[index_to_render], 
    strategy_name, 
    heuristic_name, 
    generated[index_to_render],
    elapsed[index_to_render], 
    sol_lengths[index_to_render]
)

pygame 2.6.1 (SDL 2.28.4, Python 3.13.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


2025-02-23 18:19:29.239 Python[22878:638919] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


Finished executing the plan


## Save results to file

In [8]:
results_dir = os.path.join(os.getcwd(), '.results')
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

for n in range(n_trials):
    results_file = os.path.join(results_dir, f'{level_path.split("/")[-1]}_{strategy_name}_{heuristic_name}_results_{n}.txt')
    with open(results_file, 'w') as f:
        f.write(",".join(map(str, plans[n])))

